In [7]:
import requests
from bs4 import BeautifulSoup
from traceback import format_exc
import pprint

In [2]:
def fetch_and_parse_article(url: str) -> dict:
    """
    Fetch a URL and return parsed HTML content.
    
    Args:
        url (str): The URL to fetch
        
    Returns:
        dict: Contains 'title', 'text', 'html', 'status', and 'error' (if any)
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Remove script and style elements
        for script in soup(["script", "style"]):
            script.decompose()
        
        # Extract title
        title = soup.find('title')
        title_text = title.string if title else "No title found"
        
        # Extract main content (article text)
        # Try common article containers
        article = soup.find('article') or soup.find('div', class_='article-content') or soup.find('div', class_='content')
        
        if article:
            text = article.get_text(separator='\n', strip=True)
        else:
            text = soup.get_text(separator='\n', strip=True)
        
        # Clean up whitespace
        text = '\n'.join(line.strip() for line in text.split('\n') if line.strip())
        
        return {
            'url': url,
            'title': title_text,
            'text': text,
            'html': response.content.decode('utf-8'),
            'status': response.status_code,
            'error': None
        }
    
    except requests.exceptions.RequestException as e:
        return {
            'url': url,
            'title': None,
            'text': None,
            'html': None,
            'status': None,
            'error': str(e)
        }


In [3]:
url = "https://indianexpress.com/article/smart-stocks/azad-engineering-rolls-royce-boeings-indian-supplier-10343984/"
# Usage example:
result = fetch_and_parse_article(url)
if result['error']:
    print(f"Error: {result['error']}")
else:
    print(f"Title: {result['title']}")
    print(f"Content:\n{result['text']}")

Title: Azad Engineering: Rolls-Royce and Boeing’s go-to Indian supplier — But is it overvalued? | Smart Stocks News - The Indian Express
Content:
Azad Engineering: Rolls-Royce and Boeing’s go-to Indian supplier — But is it overvalued? | Smart Stocks News - The Indian Express
skip to content
Sections
English
தமிழ்
বাংলা
മലയാളം
ગુજરાતી
हिंदी
मराठी
Business
बिज़नेस
Newsletters
The Indian Express Newsletter. (Opens in new window)
.(Opens in new window)
.(Opens in new window)
.(Opens in new window)
.(Opens in new window)
Edition
India
International
.(Open in new tab)
ePaper
.Opens in new window
Today’s Paper
.Opens in new window
Journalism of Courage
Home
.Opens in new window
ePaper
.Opens in new window
India
.Opens in new window
UPSC
.Opens in new window
Premium
.Opens in new window
Entertainment
.Opens in new window
Politics
.Opens in new window
Sports
.Opens in new window
World
.Opens in new window
Explained
.Opens in new window
Opinion
.Opens in new window
Business
.Opens in new window
C

In [10]:
def get_stock_news_from_api(stock_name: str, num_results: int = 10) -> list:
    """
    Fetch news articles for a stock using NewsAPI (free tier available).
    
    Args:
        stock_name (str): Stock name or company name
        num_results (int): Number of results to fetch
        
    Returns:
        list: List of dicts with 'title', 'url', 'source', 'description', 'published_at'
    """
    
    # Get free API key from https://newsapi.org
    API_KEY = "48c51ab301094753bb46f899b6b5a103"  # Sign up free at newsapi.org
    
    url = "https://newsapi.org/v2/everything"
    
    # Search parameters
    params = {
        'q': stock_name,
        'sortBy': 'publishedAt',
        'language': 'en',
        'pageSize': num_results,
        'apiKey': API_KEY
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        if data['status'] != 'ok':
            print(f"API Error: {data.get('message', 'Unknown error')}")
            return []
        
        articles = []
        for article in data['articles']:
            articles.append({
                'title': article['title'],
                'url': article['url'],
                'source': article['source']['name'],
                'description': article['description'],
                'published_at': article['publishedAt'],
                'image': article.get('urlToImage')
            })
        
        return articles
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching news: {e}")
        return []


# Alternative: Using GNews API (simpler, no key sometimes needed)
def get_stock_news_from_gnews(stock_name: str, num_results: int = 10) -> list:
    """
    Fetch news using GNews API (faster, simpler).
    
    Args:
        stock_name (str): Stock name
        num_results (int): Number of results
        
    Returns:
        list: News articles
    """
    
    url = "https://gnews.io/api/v4/search"
    
    params = {
        'q': stock_name,
        'lang': 'en',
        'max': num_results,
        'apikey': 'db2651be6ce35c4956fbe1fc2a5a8cdb'  # Free key from https://gnews.io
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        return data.get('articles', [])
            
    except Exception as e:
        print(format_exc(e))
        print(f"Error: {e}")
        return []

In [20]:
stock = "Azad Engineering"
    
print(f"Fetching top 10 news articles for {stock}...\n")

# Using GNews (recommended - easier setup)
articles = get_stock_news_from_gnews(stock, num_results=10)
articles += get_stock_news_from_api(stock, num_results=10)

# Print the articles
if articles:
    for i, article in enumerate(articles, 1):
        print(f"{i}. {article['title']}")
        print(f"   Source: {article['source']}")
        print(f"   Published: {article.get('publishedAt', article.get('published_at', 'N/A'))}")
        print(f"   URL: {article['url']}\n")
else:
    print("No articles found.")

Fetching top 10 news articles for Azad Engineering...

1. Azad Engineering: Rolls-Royce and Boeing’s go-to Indian supplier - But is it overvalued?
   Source: {'id': '6a3aad3dbb44ac873f72b594aaea22c4', 'name': 'The Indian Express', 'url': 'https://indianexpress.com', 'country': 'in'}
   Published: 2025-11-04T01:00:00Z
   URL: https://indianexpress.com/article/smart-stocks/azad-engineering-rolls-royce-boeings-indian-supplier-10343984/

2. Stock Market Live: GIFT Nifty Hints At Higher Open; Coal India, Urban Company, M&M Shares In Focus
   Source: {'id': '73aabc646bff1356a46223f250cc6b52', 'name': 'NDTV Profit', 'url': 'https://www.ndtvprofit.com', 'country': 'in'}
   Published: 2025-11-03T02:11:42Z
   URL: https://www.ndtvprofit.com/markets/stock-market-live-updates-nse-nifty-50-bse-sensex-nifty-bank-gift-nifty-trading-levels-november-3-2025

3. Stocks to Watch Today: Tata Motors, Bank of Baroda, SBFC Finance, Mahindra Holidays, Eicher, Titagarh Rail, MapMyIndia, Astra Microwave, Zen Tec